In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# ==========================================
# CLASE: Neurona de McCulloch-Pitts (MP)
# Basada en el notebook revisado en clase
# ==========================================
class MPNeuron:
    def __init__(self):
        self.threshold = None

    def model(self, x):
        # La neurona dispara (1) si la suma de entradas >= umbral
        return (sum(x) >= self.threshold)

    def predict(self, X):
        Y = []
        for x in X:
            result = self.model(x)
            Y.append(int(result)) # Convertir True/False a 1/0
        return np.array(Y)

    def fit(self, X, Y):
        # Aprendizaje por Fuerza Bruta: probamos todos los umbrales posibles
        accuracy = {}
        # El umbral puede ir desde 0 hasta el número de características (n_features)
        max_threshold = X.shape[1]
        
        print(f"Entrenando... Probando umbrales de 0 a {max_threshold}")
        
        for th in range(max_threshold + 1):
            self.threshold = th
            Y_pred = self.predict(X)
            accuracy[th] = accuracy_score(Y_pred, Y)
        
        # Seleccionamos el umbral que dio mejor accuracy
        self.threshold = max(accuracy, key=accuracy.get)
        print(f"Mejor umbral encontrado: {self.threshold} con Accuracy: {accuracy[self.threshold]:.4f}")

Breast Cancer con Split 55/45

In [ ]:
# ==========================================
# PARTE 1: Modificación del Split (Breast Cancer)
# Instrucción: Cambiar split a 55% Train / 45% Test
# ==========================================
print("\n--- PARTE 1: DATASET BREAST CANCER (Split 55/45) ---")
from sklearn.datasets import load_breast_cancer

# 1. Cargar datos base
data = load_breast_cancer()
X = data.data
y = data.target

# 2. Binarización (Requisito de MP Neuron)
# Convertimos valores continuos a binarios usando la media como corte (Pandas cut)
X_df = pd.DataFrame(X)
X_bin = X_df.apply(lambda x: pd.cut(x, bins=2, labels=[1, 0]), axis=0).values

# 3. SPLIT SOLICITADO: 55% Train, 45% Test
# 'stratify=y' mantiene la proporción de clases
X_train, X_test, y_train, y_test = train_test_split(
    X_bin, y, 
    test_size=0.45, 
    train_size=0.55, 
    stratify=y, 
    random_state=42
)

print(f"Datos de Entrenamiento: {X_train.shape[0]} muestras (55%)")
print(f"Datos de Prueba: {X_test.shape[0]} muestras (45%)")

# 4. Entrenar y Evaluar
mp_neuron = MPNeuron()
mp_neuron.fit(X_train, y_train)

y_pred_test = mp_neuron.predict(X_test)
acc = accuracy_score(y_test, y_pred_test)
print(f"Accuracy en Test (Split 55/45): {acc:.4f}")

NSL-KDD Procesado

In [ ]:
# ==========================================
# PARTE 2: IMPLEMENTACIÓN CON NSL-KDD
# Instrucción: Procesar y usar NSL-KDD en la neurona MP
# ==========================================
print("\n" + "="*50)
print("--- PARTE 2: PROCESAMIENTO Y USO DE NSL-KDD ---")
print("="*50)

# 1. Cargar Datasets
# Asegúrate de subir los archivos kdd_train.csv y kdd_test.csv
try:
    df_train = pd.read_csv('kdd_train.csv')
    df_test = pd.read_csv('kdd_test.csv')
    print("Datasets cargados correctamente.")
except:
    print("ERROR: Sube los archivos kdd_train.csv y kdd_test.csv al entorno.")

# 2. Preprocesamiento: Binarización del Objetivo (Target)
# 'normal' -> 0, Cualquier ataque -> 1
y_train_kdd = df_train['labels'].apply(lambda x: 0 if x == 'normal' else 1).values
y_test_kdd = df_test['labels'].apply(lambda x: 0 if x == 'normal' else 1).values

# Eliminamos la columna labels original para no usarla como feature
X_train_raw = df_train.drop('labels', axis=1)
X_test_raw = df_test.drop('labels', axis=1)

# 3. Preprocesamiento: Codificación One-Hot para Categóricas
categorical_cols = ['protocol_type', 'service', 'flag']
X_train_enc = pd.get_dummies(X_train_raw, columns=categorical_cols)
X_test_enc = pd.get_dummies(X_test_raw, columns=categorical_cols)

# ALINEACIÓN DE COLUMNAS (Crucial):
# El set de prueba puede tener menos servicios que el de entrenamiento o viceversa.
# Reindexamos el test para que tenga EXACTAMENTE las mismas columnas que el train.
X_test_enc = X_test_enc.reindex(columns=X_train_enc.columns, fill_value=0)

print(f"Dimensiones finales (Features): {X_train_enc.shape[1]}")

# 4. Preprocesamiento: Binarización de Entradas (Input Binarization)
# La Neurona MP necesita 0s y 1s.
# Estrategia: Si valor > media de la columna en train -> 1, sino -> 0
print("Binarizando entradas (Threshold = Media)...")
train_means = X_train_enc.mean()

X_train_bin = (X_train_enc > train_means).astype(int).values
X_test_bin = (X_test_enc > train_means).astype(int).values

# 5. Entrenar Neurona MP con NSL-KDD
# Usamos un subset para entrenamiento rápido (la fuerza bruta es lenta con muchas columnas)
subset_size = 2000 
print(f"Entrenando con subset de {subset_size} muestras para eficiencia...")

mp_kdd = MPNeuron()
mp_kdd.fit(X_train_bin[:subset_size], y_train_kdd[:subset_size])

# 6. Evaluación en el Test Set completo de KDD
print("Evaluando en KDD Test completo...")
y_pred_kdd = mp_kdd.predict(X_test_bin)
acc_kdd = accuracy_score(y_test_kdd, y_pred_kdd)

print(f"\nRESULTADO FINAL NSL-KDD:")
print(f"Accuracy de la Neurona MP: {acc_kdd:.4f}")